In [96]:
from prophet import Prophet

import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler

import random
from copy import deepcopy
import math

In [19]:
train_df=pd.read_csv('./data/train.csv')
test_df=pd.read_csv('./data/test.csv')

In [20]:
df=train_df[['date','rental']].copy()
df.columns=['ds','y']
m = Prophet()
m.daily_seasonality=True
m.weekly_seasonality=True
m.yearly_seasonality=True
m.fit(df)
future=m.make_future_dataframe(365)
forecast = m.predict(future)

Initial log joint probability = -73.9282
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1699.14   5.17586e-06       69.4858      0.7698      0.7698      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     110       1699.17   0.000410967       93.1272   3.446e-06       0.001      181  LS failed, Hessian reset 
     150       1699.19   3.83621e-05       77.7678   4.552e-07       0.001      266  LS failed, Hessian reset 
     177       1699.19    1.5188e-07       69.0376      0.2292           1      304   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


In [21]:
train_df['rental']=train_df['rental']/forecast[:-365]['trend']

In [22]:
train_df['date']=pd.to_datetime(train_df['date'])
train_df['month']=train_df['date'].dt.month
train_df['day']=train_df['date'].dt.day
train_df = train_df[['date', 'precipitation', 'temp_mean', 'temp_highest', 'temp_lowest',
       'PM10', 'PM2.5', 'humidity', 'sunshine_sum', 'sunshine_rate',
       'wind_mean', 'wind_max', 'month', 'day', 'rental']]

In [23]:
# 옵션에 따라, 선택한 데이터프레임의 컬럼의 결측값을 채워주는 함수
# option = 0, min, max, mean, median
def cal(my_df, column_name, option):
    df=my_df.copy()
    df['date']=pd.to_datetime(df['date'])
    df['month']=df['date'].dt.month
    nn_df = df.iloc[df[column_name].dropna().index]
    month_df_list=[nn_df[nn_df['month']==i] for i in range(1,13)]
    
    if option == 0:
        replace_list=[0 for month_df in month_df_list]
    elif option =='min':
        replace_list=[month_df[column_name].min() for month_df in month_df_list]
    elif option =='max':
        replace_list=[month_df[column_name].max() for month_df in month_df_list]
    elif option == 'mean':
        replace_list=[month_df[column_name].mean() for month_df in month_df_list]
    elif option == 'median':
        replace_list=[month_df[column_name].median() for month_df in month_df_list]
    
    replace_list = ['']+ replace_list
    
    result=[]
    for i in range(my_df.shape[0]):
        n=my_df.iloc[i][column_name]
        if math.isnan(n):
            result.append(replace_list[df.loc[i]['month']])
        else:
            result.append(n)
    my_df[column_name] = result

In [24]:
null_feature = ['precipitation','PM10','PM2.5','sunshine_sum']
best_option_group = ['min' , 'max' , 'median' , 'min']

In [25]:
for column_name, option in zip(null_feature, best_option_group):
    cal(train_df, column_name, option)

In [26]:
random_state_list=[random.randint(0, 100) for _ in range(10)]
random_state_list

[54, 7, 55, 46, 76, 88, 61, 40, 32, 79]

In [27]:
X,y = train_df.iloc[:,1:-1].to_numpy(), train_df.iloc[:,-1].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.33, random_state=False)
for i, state in enumerate(random_state_list): 
    rf = RandomForestRegressor(oob_score=True, random_state=state)
    rf.fit(X_train, y_train)
    score=rf.score(X_test, y_test)
    print(f'{i+1}번째')
    print(f'Score : {score}')
    print(f'Oob_Score : {rf.oob_score_}')

1번째
Score : 0.8084004537827776
Oob_Score : 0.8207942599133378
2번째
Score : 0.8122548160499072
Oob_Score : 0.8189813229694363
3번째
Score : 0.8131225546949846
Oob_Score : 0.8179163807125548
4번째
Score : 0.8136614613967571
Oob_Score : 0.8159725804364941
5번째
Score : 0.8106565647432986
Oob_Score : 0.8179018738370605
6번째
Score : 0.8121106194322527
Oob_Score : 0.8163139179317556
7번째
Score : 0.8122434131184917
Oob_Score : 0.8179992110506122
8번째
Score : 0.8152912377808071
Oob_Score : 0.8187398512807251
9번째
Score : 0.8115501586700198
Oob_Score : 0.8150259114837362
10번째
Score : 0.8120317529883875
Oob_Score : 0.8163940876027691


In [28]:
train_df=pd.read_csv('./data/train.csv')
test_df=pd.read_csv('./data/test.csv')

In [29]:
df=train_df[['date','rental']].copy()
df.columns=['ds','y']
m = Prophet()
m.daily_seasonality=True
m.weekly_seasonality=True
m.yearly_seasonality=True
m.fit(df)
future=m.make_future_dataframe(365)
forecast = m.predict(future)

Initial log joint probability = -73.9282
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1699.14   5.17586e-06       69.4858      0.7698      0.7698      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     110       1699.17   0.000410967       93.1272   3.446e-06       0.001      181  LS failed, Hessian reset 
     150       1699.19   3.83621e-05       77.7678   4.552e-07       0.001      266  LS failed, Hessian reset 
     177       1699.19    1.5188e-07       69.0376      0.2292           1      304   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


In [30]:
train_df['rental']=train_df['rental']/forecast[:-365]['trend']

In [31]:
train_df['date']=pd.to_datetime(train_df['date'])
train_df['year']=train_df['date'].dt.year
train_df['month']=train_df['date'].dt.month
train_df['day']=train_df['date'].dt.day
train_df = train_df[['date', 'precipitation', 'temp_mean', 'temp_highest', 'temp_lowest',
       'PM10', 'PM2.5', 'humidity', 'sunshine_sum', 'sunshine_rate',
       'wind_mean', 'wind_max', 'year', 'month', 'day', 'rental']]

In [32]:
# 옵션에 따라, 선택한 데이터프레임의 컬럼의 결측값을 채워주는 함수
# option = 0, min, max, mean, median
def cal(my_df, column_name, option):
    df=my_df.copy()
    df['date']=pd.to_datetime(df['date'])
    df['month']=df['date'].dt.month
    nn_df = df.iloc[df[column_name].dropna().index]
    month_df_list=[nn_df[nn_df['month']==i] for i in range(1,13)]
    
    if option == 0:
        replace_list=[0 for month_df in month_df_list]
    elif option =='min':
        replace_list=[month_df[column_name].min() for month_df in month_df_list]
    elif option =='max':
        replace_list=[month_df[column_name].max() for month_df in month_df_list]
    elif option == 'mean':
        replace_list=[month_df[column_name].mean() for month_df in month_df_list]
    elif option == 'median':
        replace_list=[month_df[column_name].median() for month_df in month_df_list]
    
    replace_list = ['']+ replace_list
    
    result=[]
    for i in range(my_df.shape[0]):
        n=my_df.iloc[i][column_name]
        if math.isnan(n):
            result.append(replace_list[df.loc[i]['month']])
        else:
            result.append(n)
    my_df[column_name] = result

In [33]:
null_feature = ['precipitation','PM10','PM2.5','sunshine_sum']
best_option_group = ['min' , 'max' , 'median' , 'min']

In [34]:
for column_name, option in zip(null_feature, best_option_group):
    cal(train_df, column_name, option)

In [35]:
random_state_list=[random.randint(0, 100) for _ in range(10)]
random_state_list

[10, 12, 49, 96, 85, 78, 58, 0, 72, 15]

In [36]:
X,y = train_df.iloc[:,1:-1].to_numpy(), train_df.iloc[:,-1].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.33, random_state=False)
for i, state in enumerate(random_state_list): 
    rf = RandomForestRegressor(oob_score=True, random_state=state)
    rf.fit(X_train, y_train)
    score=rf.score(X_test, y_test)
    print(f'{i+1}번째')
    print(f'Score : {score}')
    print(f'Oob_Score : {rf.oob_score_}')

1번째
Score : 0.9005671394042061
Oob_Score : 0.907286205250915
2번째
Score : 0.905411105656172
Oob_Score : 0.9076086471471715
3번째
Score : 0.9053814014075534
Oob_Score : 0.9071575176385851
4번째
Score : 0.9022160481419764
Oob_Score : 0.905506108058991
5번째
Score : 0.9029218612393313
Oob_Score : 0.9057274426533133
6번째
Score : 0.9045051471572381
Oob_Score : 0.9036948552382839
7번째
Score : 0.9017862794865743
Oob_Score : 0.90664721656037
8번째
Score : 0.9047378637168084
Oob_Score : 0.9086901864732576
9번째
Score : 0.9019144951486288
Oob_Score : 0.9112685461772113
10번째
Score : 0.9025770434085442
Oob_Score : 0.9079712744580146


In [38]:
print(*sorted(zip(train_df.iloc[:,1:-1].columns ,rf.feature_importances_), key=lambda x : x[1], reverse=True), sep='\n')

('temp_highest', 0.5465225671813158)
('year', 0.11812816652396348)
('humidity', 0.08821519125588559)
('precipitation', 0.08671087213227834)
('sunshine_sum', 0.030216469832044124)
('temp_mean', 0.02902691484394027)
('sunshine_rate', 0.024540564865326416)
('temp_lowest', 0.02419288935508021)
('month', 0.014811920291157082)
('PM2.5', 0.009354495564610432)
('PM10', 0.009329028255293878)
('wind_max', 0.006580866104073585)
('day', 0.006396008468677072)
('wind_mean', 0.0059740453263537364)


In [40]:
importance_list = sorted(zip(train_df.iloc[:,1:-1].columns ,rf.feature_importances_), key=lambda x : x[1], reverse=True)
importance_list[0]

('temp_highest', 0.5465225671813158)

In [56]:
for i in range(1, 10):
    drop_train_df = train_df.drop([*[col for col, acc in importance_list[-i:]]], axis=1)
    X,y = drop_train_df.iloc[:,1:-1].to_numpy(), drop_train_df.iloc[:,-1].to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.33, random_state=False)
    rf = RandomForestRegressor(oob_score=True, random_state=state)
    rf.fit(X_train, y_train)
    score=rf.score(X_test, y_test)
    print(i, score, X.shape)
    if i ==4:
        break

1 0.9007463645962364 (1095, 13)
2 0.9019445365569001 (1095, 12)
3 0.9038751581664527 (1095, 11)
4 0.9040955714379885 (1095, 10)


In [58]:
for i in range(1, 10):
    drop_train_df = train_df.drop([*[col for col, acc in importance_list[-i:]]], axis=1)
    drop_train_df.drop(['year'], axis=1, inplace=True)
    X,y = drop_train_df.iloc[:,1:-1].to_numpy(), drop_train_df.iloc[:,-1].to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.33, random_state=False)
    rf = RandomForestRegressor(oob_score=True, random_state=state)
    rf.fit(X_train, y_train)
    score=rf.score(X_test, y_test)
    print(i, score, X.shape)
    if i ==4:
        break

1 0.802787187502729 (1095, 12)
2 0.8023627531071444 (1095, 11)
3 0.7837700053396297 (1095, 10)
4 0.7778688962626799 (1095, 9)


In [57]:
importance_list[-4:]

[('PM10', 0.009329028255293878),
 ('wind_max', 0.006580866104073585),
 ('day', 0.006396008468677072),
 ('wind_mean', 0.0059740453263537364)]

In [43]:
drop_train_df_1=train_df.drop([*[col for col, acc in importance_list[-3:]]], axis=1)

In [166]:
from supervised.automl import BaseAutoML, AutoML

from prophet import Prophet

import pandas as pd

import math

In [167]:
train_df=pd.read_csv('./data/train.csv')
test_df=pd.read_csv('./data/test.csv')

In [168]:
df=train_df[['date','rental']].copy()
df.columns=['ds','y']
m = Prophet()
m.daily_seasonality=True
m.weekly_seasonality=True
m.yearly_seasonality=True
m.fit(df)
future=m.make_future_dataframe(365)
forecast = m.predict(future)

Initial log joint probability = -73.9282
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       1699.14   5.17586e-06       69.4858      0.7698      0.7698      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     110       1699.17   0.000410967       93.1272   3.446e-06       0.001      181  LS failed, Hessian reset 
     150       1699.19   3.83621e-05       77.7678   4.552e-07       0.001      266  LS failed, Hessian reset 
     177       1699.19    1.5188e-07       69.0376      0.2292           1      304   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


In [169]:
train_df['rental']=train_df['rental']/forecast[:-365]['trend']

In [170]:
train_df['date']=pd.to_datetime(train_df['date'])
train_df['year']=train_df['date'].dt.year
train_df['month']=train_df['date'].dt.month
train_df['day']=train_df['date'].dt.day
train_df['weekday'] = train_df['date'].dt.dayofweek
train_df = pd.get_dummies(data = train_df, prefix=['year'], columns=['year'])
train_df['year']=train_df['date'].dt.year

In [171]:
train_df.columns

Index(['date', 'precipitation', 'temp_mean', 'temp_highest', 'temp_lowest',
       'PM10', 'PM2.5', 'humidity', 'sunshine_sum', 'sunshine_rate',
       'wind_mean', 'wind_max', 'rental', 'year', 'month', 'day'],
      dtype='object')

In [172]:
train_df

,date,precipitation,temp_mean,temp_highest,temp_lowest,PM10,PM2.5,humidity,sunshine_sum,sunshine_rate,wind_mean,wind_max,rental,year,month,day
0,2018-01-01,NaN,-1.3,3.8,-5.1,34.0,17.0,39.1,8.3,86.5,1.4,3.8,0.599347,2018,1,1
1,2018-01-02,NaN,-1.8,1.8,-4.3,36.0,22.0,42.0,7.9,82.3,1.8,4.9,0.861322,2018,1,2
2,2018-01-03,NaN,-4.7,-0.4,-7.1,31.0,19.0,42.3,8.6,88.7,2.2,3.5,0.861039,2018,1,3
3,2018-01-04,NaN,-4.7,-0.7,-8.7,39.0,24.0,43.0,6.2,63.9,1.4,3.5,0.851881,2018,1,4
4,2018-01-05,NaN,-3.0,1.6,-5.6,51.0,35.0,48.4,8.2,84.5,1.7,3.6,0.921342,2018,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,2020-12-27,0.0,5.8,10.0,1.4,70.0,42.0,62.9,5.9,61.5,1.8,2.8,0.631485,2020,12,27
1091,2020-12-28,1.3,6.7,11.4,4.2,66.0,44.0,72.1,8.0,83.3,1.4,3.1,0.797950,2020,12,28
1092,2020-12-29,0.2,0.1,4.3,-6.2,69.0,46.0,70.8,0.0,0.0,2.9,6.1,0.607672,2020,12,29
1093,2020-12-30,NaN,-10.9,-6.2,-12.9,39.0,15.0,55.5,8.3,86.5,4.1,6.2,0.382048,2020,12,30


In [173]:
train_df = train_df[['date', 'precipitation', 'temp_mean', 'temp_highest', 'temp_lowest',
       'PM10', 'PM2.5', 'humidity', 'sunshine_sum', 'sunshine_rate',
       'wind_mean', 'wind_max', 'year', 'month', 'day', 'weekday', 'rental']]

KeyError: "['weekday'] not in index"

In [174]:
# 옵션에 따라, 선택한 데이터프레임의 컬럼의 결측값을 채워주는 함수
# option = 0, min, max, mean, median
def cal(my_df, column_name, option):
    df=my_df.copy()
    df['date']=pd.to_datetime(df['date'])
    df['month']=df['date'].dt.month
    nn_df = df.iloc[df[column_name].dropna().index]
    month_df_list=[nn_df[nn_df['month']==i] for i in range(1,13)]
    
    if option == 0:
        replace_list=[0 for month_df in month_df_list]
    elif option =='min':
        replace_list=[month_df[column_name].min() for month_df in month_df_list]
    elif option =='max':
        replace_list=[month_df[column_name].max() for month_df in month_df_list]
    elif option == 'mean':
        replace_list=[month_df[column_name].mean() for month_df in month_df_list]
    elif option == 'median':
        replace_list=[month_df[column_name].median() for month_df in month_df_list]
    
    replace_list = ['']+ replace_list
    
    result=[]
    for i in range(my_df.shape[0]):
        n=my_df.iloc[i][column_name]
        if math.isnan(n):
            result.append(replace_list[df.loc[i]['month']])
        else:
            result.append(n)
    my_df[column_name] = result

In [175]:
null_feature = ['precipitation','PM10','PM2.5','sunshine_sum']
best_option_group = ['min' , 'max' , 'median' , 'min']
for column_name, option in zip(null_feature, best_option_group):
    cal(train_df, column_name, option)

In [176]:
random_state_list=[random.randint(0, 100) for _ in range(10)]
random_state_list

[99, 14, 43, 61, 77, 34, 53, 94, 8, 8]

In [177]:
# train_df.drop([*[col for col, acc in importance_list[-i:]]], axis=1, inplace=True)

# train_df.drop(['year'], axis=1, inplace=True)

X,y = train_df.iloc[:,1:-1].to_numpy(), train_df.iloc[:,-1].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.33, random_state=False)

In [179]:
automl = AutoML(mode="Explain", n_jobs=-1, results_path='./hojun_a_explain5')
automl.fit(X_train, y_train)

AutoML directory: ./hojun_a_explain5
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Baseline', 'Linear', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_Baseline rmse 8.581645 trained in 0.48 seconds
2_DecisionTree rmse 8.531719 trained in 2.95 seconds
3_Linear rmse 9.467381 trained in 1.72 seconds
* Step default_algorithms will try to check up to 3 models
4_Default_Xgboost rmse 9.088538 trained in 2.52 seconds
5_Default_NeuralNetwork rmse 9.475592 trained in 0.67 seconds
6_Default_RandomForest rmse 9.26451 trained in 3.02 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 8.486422 trained in 0.17 seconds
AutoML fit time: 15.33 seconds
AutoML best model: Ensemble


AutoML(results_path='./hojun_a_explain5')

In [180]:
model=AutoML(results_path='./hojun_a_explain5')

In [181]:
r2_score(y_test, model.predict(X_test))

-0.013420770636582757

In [164]:
r2_score(y_test, model.predict(X_test))

X has feature names, but StandardScaler was fitted without feature names


0.9177438607572415

In [150]:
r2_score(y_test, model.predict(X_test))

X has feature names, but StandardScaler was fitted without feature names


0.9127770445025871

In [82]:
r2_score(y_test, model.predict(X_test))

X has feature names, but StandardScaler was fitted without feature names


0.917635727706018

In [13]:
r2_score(y_test, model.predict(X_test))

X has feature names, but StandardScaler was fitted without feature names


0.9636881291772562

In [98]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(train_df.iloc[:,1:-1])

In [100]:
scaled.shape

(1095, 13)

In [102]:
X,y = scaled, train_df.iloc[:,-1].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.33, random_state=False)

In [103]:
automl = AutoML(mode="Explain", n_jobs=-1, results_path='./hojun_a_explain2_scaled')
automl.fit(X_train, y_train)

AutoML directory: ./hojun_a_explain2_scaled
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Baseline', 'Linear', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network']
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble']
* Step simple_algorithms will try to check up to 3 models
1_Baseline rmse 0.775359 trained in 0.41 seconds
2_DecisionTree rmse 0.472874 trained in 8.62 seconds
3_Linear rmse 0.478638 trained in 1.68 seconds
* Step default_algorithms will try to check up to 3 models
4_Default_Xgboost rmse 0.360609 trained in 2.62 seconds
5_Default_NeuralNetwork rmse 0.383688 trained in 0.68 seconds
6_Default_RandomForest rmse 0.403028 trained in 2.46 seconds
* Step ensemble will try to check up to 1 model
Ensemble rmse 0.348039 trained in 0.17 seconds
AutoML fit time: 20.6 seconds
AutoML best model: Ensemble


AutoML(results_path='./hojun_a_explain2_scaled')

In [104]:
model=AutoML(results_path='./hojun_a_explain2_scaled')

In [105]:
r2_score(y_test, model.predict(X_test))

X has feature names, but StandardScaler was fitted without feature names


0.8091671317940039

In [ ]:
ㅋ